In [1]:
""" https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf
https://user.ceng.metu.edu.tr/~emre/resources/courses/AdvancedDL_Spring2017/DQN_Muhammed.pdf
https://www.youtube.com/watch?v=NP8pXZdU-5U&t=1s """
import matplotlib.pyplot as plt
import torch
import numpy as np
from copy import deepcopy
from tqdm import tqdm
from game import *
from Agent import *
import collections
import random
from itertools import count

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
BATCH_SIZE = 64
GAMMA = 0.999
EPS_START = 1.0
EPS_END = 0.001
EPS_DECAY = 0.001
TARGET_UPDATE = 10
MY_SEED = 143
LR = 5e-4
MEMORY_SIZE = 10000
# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
screen_height, screen_width = 84, 84  # Defining as grid size in snake
nr_actions = 4

agent_007 = Agent(gamma=GAMMA,exploration_rate=EPS_START,lr=LR,
                  input_dims=(screen_height,screen_width), batch_size=BATCH_SIZE,
                  nr_actions=nr_actions, max_mem_size=MEMORY_SIZE,
                  nr_consecutive_frames=1, exploration_rate_min=EPS_END,
                  exploration_decay_rate=EPS_DECAY,seed=MY_SEED)


In [3]:
NUM_EPISODES = 100

episode_durations = []
episode_rewards = []
episode_exploration_rates = []

t_total_counter = 0
for i_episode in tqdm(range(NUM_EPISODES)):
    # Initialize the environment and state
    game_env = SimpleSnakeApp(seed=MY_SEED + i_episode,  # Try both constant seed and varying seed for each episode
                              display_gameplay=False)
    current_game_score = 0
    current_game_duration = 0

    current_exploration_rate = 0
    current_steps_counter = 0
    game_over = False
    current_state = game_env.get_state()

    for t in count():
        # Select and perform an action according to greedy epsilon strategy
        action = agent_007.choose_action(current_state)

        # Perform action in game environment and observe reward and new state + termination flag.
        reward, game_over, new_observation = game_env.step(action)
        current_game_score += reward
        reward = torch.tensor([reward])

        # Observe new state
        last_state = current_state
        current_state = new_observation

        # Store the transition in memory
        # print(type(last_state), type(action),type(reward),type(current_state),type(game_over))
        agent_007.store_transition(state=last_state,action=action,
                                   reward=reward,new_state=current_state,
                                   done_flag=game_over)



        # Perform one step of the optimization (on the policy network)
        agent_007.learn(t_total_counter)
        if game_over:
            break
        t_total_counter += 1
        current_game_duration += 1

        # Update the target network, copying all weights and biases in DQN from policy to target network
        if t % TARGET_UPDATE == 0:
            agent_007.align_networks()

    episode_rewards.append(current_game_score)
    episode_durations.append(current_game_duration)
    episode_exploration_rates.append(agent_007.exploration_rate)


  0%|          | 0/100 [00:00<?, ?it/s]

Apple hit


  0%|          | 0/100 [01:05<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(14,4))
ax[0].plot([ i for i in range(len(episode_rewards))], episode_rewards)
ax[1].plot([ i for i in range(len(episode_durations))], episode_durations)
ax[2].plot([ i for i in range(len(episode_exploration_rates))], episode_exploration_rates)
plt.show()

In [ ]:
test = 2, "b":3}
test["a"]